In [1]:
# Import libraries:
import pandas as pd
import importlib 
import dt_ids7_export_utils as bh_utils
import rad_xa_mapping_func as bh_rad_xa
import plot_functions as bh_plot
import reporting_utils as bh_report

# Make sure personal libraries are reloaded incase of adjustments:
importlib.reload(bh_utils)
importlib.reload(bh_rad_xa)
importlib.reload(bh_plot)
importlib.reload(bh_report)

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path

# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)
data = bh_utils.merge_ids7_dt(df_ids7, df_dt, True)
del df_dt, df_ids7

Number of rows with NaT in the column "Bestilt dato og tidspunkt": 1
Number of cancelled procedures: 419
Number of non-human subjects: 8
Number of rows with invalid accession number: 0
Number of accession numbers in IDS7: 2371
Number of accession numbers in IDS7 not in DoseTrack: 142
Inserted accession number: NRRH000008244732 for patient: PAS0246, time: 2023-05-18 13:00:00, accession numbers: ['NRRH000008244732' 'NRRH000008244733']
Inserted accession number: NRRH000007967678 for patient: PAS1308, time: 2023-01-03 07:00:00, accession numbers: ['NRRH000007967670' 'NRRH000007967678']
The accession numbers have been changed. Running check_accession_ids7_vs_dt
Number of accession numbers in IDS7: 2369
Number of accession numbers in IDS7 not in DoseTrack: 140
Number of accession numbers in DoseTrack: 2490
Number of accession numbers in DoseTrack not in IDS7: 261
This column will not be included in the merged data.


The IDS7 and DoseTrack has merged data of length: 2229


In [2]:
data = bh_rad_xa.map_rad_xa_procedures(data, True)

Mapping procedures...

Nefrostomi & innleggelse av dren -> Nefrostomi innleggelse
WARNING! No procedures were targeted by this mapping!


Nefrostomi & ~innleggelse av dren -> Nefrostomi skifte eller fjerning


In [35]:
key = 'Nefrostomi & Nyrer & ureter'
excl = 'Innleggelse & skifte & dren'
inc_key_list = key.split(' & ')
excl_key_list = excl.split(' & ')

beskrivelse = 'Nefrostomi, nyrer, Ureter, Innleggelse'

In [37]:
# Check if al elements in the key_list is in the beskrivelse:
if all(elem.lower() in beskrivelse.lower() for elem in inc_key_list) & ~any(element.lower() in beskrivelse.lower() for element in excl_key_list):
    print('Criteria satisfied')
else:
    print('Criteria not satisfied')

Criteria not satisfied


In [ ]:
# put all elements in the key_list that does not start with ~ in a list
include_list = [x for x in key_list if not x.startswith('~')]
# put all elements in the key_list that starts with ~ in a list
exclude_list = [x[1:] for x in key_list if x.startswith('~')]

In [ ]:
data['Mapping Target'] = False
data.loc[data['Beskrivelse'].apply(lambda x: all(substring in x for substring in include_list)) &\
        ~data['Beskrivelse'].str.contains('|'.join(exclude_list)), 'Mapping Target'] = True

In [ ]:
bh_utils.export_examination_codes_to_text_file(data)

In [ ]:
bh_utils.delete_reports(True)